In [1]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

if torch.cuda.is_available():
    print("Yeah we have a GPU!")
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

import torchvision
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import itertools
import time
import gc

from gdeep.decision_boundary import GradientFlowDecisionBoundaryCalculator, QuasihyperbolicDecisionBoundaryCalculator

Yeah we have a GPU!


# Train CNN-Autoencoder

In [2]:
# https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/conv_autoencoder.py


if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 12),
            nn.ReLU(True),
            nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model_auto = autoencoder().to(dev)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model_auto.parameters(), lr=learning_rate, weight_decay=1e-5)

In [3]:
"""
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).to(dev)
        # ===================forward=====================
        output = model_auto(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, 'mlp_img/image_{}.png'.format(epoch))

torch.save(model_auto, 'sim_autoencoder.pth')
"""

"\nfor epoch in range(num_epochs):\n    for data in dataloader:\n        img, _ = data\n        img = img.view(img.size(0), -1)\n        img = Variable(img).to(dev)\n        # ===================forward=====================\n        output = model_auto(img)\n        loss = criterion(output, img)\n        # ===================backward====================\n        optimizer.zero_grad()\n        loss.backward()\n        optimizer.step()\n    # ===================log========================\n    print('epoch [{}/{}], loss:{:.4f}'\n          .format(epoch + 1, num_epochs, loss.data.item()))\n    if epoch % 10 == 0:\n        pic = to_img(output.cpu().data)\n        save_image(pic, 'mlp_img/image_{}.png'.format(epoch))\n\ntorch.save(model_auto, 'sim_autoencoder.pth')\n"

In [3]:
model_auto = torch.load('sim_autoencoder.pth')
model_auto.eval()

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=12, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=12, out_features=3, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Tanh()
  )
)

In [5]:
embedding_df = pd.DataFrame(columns=list('xyzc'))

for x, y in itertools.islice(dataloader, 20):
    embedded_batch = model_auto.encoder(x.view(x.size(0), -1)).detach().numpy()
    embedded_batch_df = pd.DataFrame(
        np.concatenate((embedded_batch, y.unsqueeze(1).numpy()), 1),
        columns=list('xyzc'))
    embedding_df = embedding_df.append(
        embedded_batch_df,
        ignore_index=True)

In [6]:
"""
fig = px.scatter_3d(embedding_df, x='x', y='y', z='z',
              color='c')
fig.show()
"""

"\nfig = px.scatter_3d(embedding_df, x='x', y='y', z='z',\n              color='c')\nfig.show()\n"

In [4]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


transform=transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,), (0.3081,))
    ])

batch_size = 64
test_batch_size = 64
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

dataset1 = datasets.MNIST('./data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
"""
seed = 42
torch.manual_seed(seed)
batch_size = 64
test_batch_size = 64
lr = 0.01
gamma = 0.7
epochs = 14
log_interval = 10

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

transform=transforms.Compose([
    transforms.ToTensor()
    ])


model = Net().to(dev)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(log_interval, model, dev, train_loader, optimizer, epoch)
    test(model, dev, test_loader)
    scheduler.step()
"""

"\nseed = 42\ntorch.manual_seed(seed)\nbatch_size = 64\ntest_batch_size = 64\nlr = 0.01\ngamma = 0.7\nepochs = 14\nlog_interval = 10\n\ntrain_kwargs = {'batch_size': batch_size}\ntest_kwargs = {'batch_size': test_batch_size}\n\ntransform=transforms.Compose([\n    transforms.ToTensor()\n    ])\n\n\nmodel = Net().to(dev)\noptimizer = optim.Adadelta(model.parameters(), lr=lr)\n\nscheduler = StepLR(optimizer, step_size=1, gamma=gamma)\nfor epoch in range(1, epochs + 1):\n    train(log_interval, model, dev, train_loader, optimizer, epoch)\n    test(model, dev, test_loader)\n    scheduler.step()\n"

In [5]:
#torch.save(model, '/home/reinauer/EPFL/giotto-deep/examples/cnn_minst_10.pth')
model = torch.load('cnn_minst_10.pth')
model.eval()

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [8]:
class LambdaLayer(nn.Module):
    def __init__(self, lam):
        super().__init__()
        self.lam = lam
    def forward(self, x):
        return self.lam(x)

class DBMulticlass(nn.Module):
    def forward(self, x):
        y = torch.topk(x, 4).values
        return (y[:, 0] - y[:, 3]) + 0.5

embedding_model_strat = nn.Sequential(model_auto.decoder,
                        LambdaLayer(lambda x: x.reshape(-1, 1, 28, 28)),
                        model,
                        nn.Softmax(),
                        DBMulticlass())

embedding_model_prob = nn.Sequential(model_auto.decoder, 
                                     LambdaLayer(lambda x: x.reshape(-1, 1, 28, 28)),
                                     model,
                                     nn.Softmax())

In [12]:
embedding_model_prob(torch.rand((10, 3)))

tensor([[6.3769e-03, 1.3862e-03, 3.0843e-04, 5.2468e-03, 8.7423e-04, 9.7353e-01,
         2.5234e-03, 3.9665e-03, 1.1613e-03, 4.6293e-03],
        [8.2451e-03, 1.5615e-03, 3.9490e-04, 5.2104e-03, 9.0703e-04, 9.7040e-01,
         2.5090e-03, 4.8394e-03, 1.1421e-03, 4.7929e-03],
        [3.0700e-02, 2.4252e-03, 1.6461e-03, 9.9389e-03, 1.9218e-03, 9.1616e-01,
         4.8395e-03, 1.9203e-02, 1.3084e-03, 1.1862e-02],
        [2.9291e-02, 2.7858e-03, 1.5112e-03, 1.0486e-02, 1.4897e-03, 9.2206e-01,
         4.0806e-03, 1.6302e-02, 1.2648e-03, 1.0726e-02],
        [7.4159e-03, 1.2680e-03, 3.5179e-04, 5.0203e-03, 1.0316e-03, 9.6855e-01,
         3.0822e-03, 5.4678e-03, 1.2121e-03, 6.5993e-03],
        [2.7018e-02, 2.9398e-03, 1.4210e-03, 1.1385e-02, 1.4405e-03, 9.2305e-01,
         4.0667e-03, 1.6275e-02, 1.2664e-03, 1.1137e-02],
        [4.0788e-03, 1.2894e-03, 2.2330e-04, 5.7224e-03, 6.2610e-04, 9.7717e-01,
         2.2369e-03, 3.2216e-03, 1.0366e-03, 4.3903e-03],
        [2.9715e-02, 2.1789

In [19]:
"""
n_delta = 20
t = np.linspace(-10.5, 10.5, n_delta)
x, y, z = np.meshgrid(t,t,t)
xyz = torch.from_numpy(np.stack((x, y, z), axis=-1)).reshape(-1, 3)
xyz = xyz.type(torch.float32)
values = embedding_model(xyz).reshape(n_delta, n_delta, n_delta,-1).detach()
   
trace_1 = go.Isosurface(
    x = x.flatten(),
    y = y.flatten(),
    z = z.flatten(),
    value=values.flatten(),
    isomin=0.0,
    isomax=0.1)

fig = go.FigureWidget([trace_1])
fig.show()
"""

'\nn_delta = 20\nt = np.linspace(-10.5, 10.5, n_delta)\nx, y, z = np.meshgrid(t,t,t)\nxyz = torch.from_numpy(np.stack((x, y, z), axis=-1)).reshape(-1, 3)\nxyz = xyz.type(torch.float32)\nvalues = embedding_model(xyz).reshape(n_delta, n_delta, n_delta,-1).detach()\n   \ntrace_1 = go.Isosurface(\n    x = x.flatten(),\n    y = y.flatten(),\n    z = z.flatten(),\n    value=values.flatten(),\n    isomin=0.0,\n    isomax=0.1)\n\nfig = go.FigureWidget([trace_1])\nfig.show()\n'

In [25]:
torch.cuda.empty_cache()

embedding_model.to(dev)
embedding_model.eval()

Sequential(
  (0): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Tanh()
  )
  (1): LambdaLayer()
  (2): Net(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (dropout1): Dropout(p=0.25, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
    (fc1): Linear(in_features=9216, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
  )
  (3): DBMulticlass()
)

In [26]:
#del initial_points_batch
torch.cuda.empty_cache()

In [13]:
n_steps = 1000
precision = 0.01
range_db = 70.

n_batches = 2#60*12#100
batch_size = 200#1500#4000

# Initialize DataFrame
db_sample_df = pd.DataFrame(columns = ["x"+str(i) for i in range(3)])

for b in range(n_batches):
    tic = time.perf_counter()
    # Sample points on decision boundary
    initial_points_batch = range_db * torch.rand((batch_size, 3)) - range_db/2.
    initial_points_batch = initial_points_batch.to(dev, non_blocking=True)
    g = GradientFlowDecisionBoundaryCalculator(
                model=embedding_model,
                initial_points=initial_points_batch,
                optimizer=lambda params: torch.optim.Adam(params)
    )
    g.step(n_steps)
    
    # Save computed points to DataFrame
    db_sample_tensor = g.get_filtered_decision_boundary(precision)
    
    median = torch.median(embedding_model(db_sample_tensor) - 0.5).cpu().detach().numpy()
    db_sample = db_sample_tensor.to(device='cpu').detach().numpy()
    db_sample_df_batch = pd.DataFrame(db_sample, columns = ["x"+str(i) for i in range(3)])
    
    db_sample_df = db_sample_df.append(db_sample_df_batch, ignore_index=True)
    
    # Delete tensors on cuda
    del db_sample_tensor
    del initial_points_batch
    del db_sample
    torch.cuda.empty_cache()
    toc = time.perf_counter()
    print(f'batch: {b+1}/{n_batches} \t\
    computation time/batch: {toc - tic:0.4f}sec\t\
    number of points: {db_sample_df_batch.shape[0]}\n\t\
    median distance2db: {median}')
    del db_sample_df_batch

batch: 1/2 	    computation time/batch: 9.4228sec	    number of points: 31
	    median distance2db: 3.838539123535156e-05


KeyboardInterrupt: 

In [27]:
n_steps = 100
precision = 0.01
range_db = 70.

n_batches = 1000
batch_size = 3000#4000

# Initialize DataFrame
db_sample_df = pd.DataFrame(columns = ["x"+str(i) for i in range(3)])

for b in range(n_batches):
    tic = time.perf_counter()
    # Sample points on decision boundary
    initial_points_batch = range_db * torch.rand((batch_size, 3)) - range_db/2.
    initial_points_batch = initial_points_batch.to(dev, non_blocking=True)
    
    initial_points_batch.requires_grad = False
    delta = torch.zeros_like(initial_points_batch, requires_grad=True).to(dev)
    loss = embedding_model(initial_points_batch + delta).sum()
    loss.backward()
    initial_vectors = delta.grad.detach()
    
    
    g = QuasihyperbolicDecisionBoundaryCalculator(
                model=embedding_model,
                initial_points=initial_points_batch,
                initial_vectors=initial_vectors,
                integrator=None
    )
    g.step(n_steps)
    
    # Save computed points to DataFrame
    db_sample_tensor = g.get_filtered_decision_boundary(precision)
    toc = time.perf_counter()
    print(f'computation DB time/batch: {toc - tic:0.4f}')
    
    median = torch.median(embedding_model(db_sample_tensor) - 0.5).cpu().detach().numpy()
    db_sample = db_sample_tensor.to(device='cpu').detach().numpy()
    db_sample_df_batch = pd.DataFrame(db_sample, columns = ["x"+str(i) for i in range(3)])
    
    db_sample_df = db_sample_df.append(db_sample_df_batch, ignore_index=True)
    
    # Delete tensors on cuda
    del db_sample_tensor
    del initial_points_batch
    del db_sample
    toc = time.perf_counter()
    print(f'batch: {b+1}/{n_batches} \t\
    computation time/batch: {toc - tic:0.4f}sec\t\
    number of points: {db_sample_df_batch.shape[0]}\n\t\
    median distance2db: {median}')
    del db_sample_df_batch
    del initial_vectors
    del g
    del delta
    torch.cuda.empty_cache()
    gc.collect()
    if b%50 == 0:
        db_sample_df.to_csv(f'computed_points_codim_3_1M_{b:04d}.csv')

computation DB time/batch: 53.8343
batch: 1/1000 	    computation time/batch: 53.8373sec	    number of points: 10
	    median distance2db: 0.0007183551788330078
computation DB time/batch: 53.9291
batch: 2/1000 	    computation time/batch: 53.9320sec	    number of points: 16
	    median distance2db: 0.0034782886505126953
computation DB time/batch: 54.0758
batch: 3/1000 	    computation time/batch: 54.0785sec	    number of points: 10
	    median distance2db: 0.002861499786376953
computation DB time/batch: 53.9366
batch: 4/1000 	    computation time/batch: 53.9402sec	    number of points: 12
	    median distance2db: 0.0023703575134277344
computation DB time/batch: 53.9631
batch: 5/1000 	    computation time/batch: 53.9659sec	    number of points: 17
	    median distance2db: 0.0026977062225341797
computation DB time/batch: 53.9973
batch: 6/1000 	    computation time/batch: 54.0005sec	    number of points: 11
	    median distance2db: 0.0010905265808105469
computation DB time/batch: 54.0053


computation DB time/batch: 53.9689
batch: 52/1000 	    computation time/batch: 53.9726sec	    number of points: 11
	    median distance2db: 0.0011227130889892578
computation DB time/batch: 53.9672
batch: 53/1000 	    computation time/batch: 53.9700sec	    number of points: 14
	    median distance2db: 0.0021833181381225586
computation DB time/batch: 53.9923
batch: 54/1000 	    computation time/batch: 53.9959sec	    number of points: 11
	    median distance2db: 0.0015270709991455078
computation DB time/batch: 53.9923
batch: 55/1000 	    computation time/batch: 53.9963sec	    number of points: 12
	    median distance2db: 0.0001443624496459961
computation DB time/batch: 53.9716
batch: 56/1000 	    computation time/batch: 53.9747sec	    number of points: 7
	    median distance2db: 0.001255035400390625
computation DB time/batch: 53.9188
batch: 57/1000 	    computation time/batch: 53.9219sec	    number of points: 9
	    median distance2db: 0.0019484758377075195
computation DB time/batch: 53.9

computation DB time/batch: 53.8063
batch: 103/1000 	    computation time/batch: 53.8100sec	    number of points: 11
	    median distance2db: 0.0030012130737304688
computation DB time/batch: 53.6167
batch: 104/1000 	    computation time/batch: 53.6199sec	    number of points: 19
	    median distance2db: 0.0010960102081298828
computation DB time/batch: 53.5526
batch: 105/1000 	    computation time/batch: 53.5560sec	    number of points: 13
	    median distance2db: 0.0018322467803955078
computation DB time/batch: 53.5599
batch: 106/1000 	    computation time/batch: 53.5635sec	    number of points: 12
	    median distance2db: 0.0036640167236328125
computation DB time/batch: 54.0839
batch: 107/1000 	    computation time/batch: 54.0866sec	    number of points: 15
	    median distance2db: 0.0010422468185424805
computation DB time/batch: 53.9697
batch: 108/1000 	    computation time/batch: 53.9725sec	    number of points: 13
	    median distance2db: 0.0021172761917114258
computation DB time/ba

computation DB time/batch: 53.9120
batch: 154/1000 	    computation time/batch: 53.9149sec	    number of points: 15
	    median distance2db: 0.002562403678894043
computation DB time/batch: 53.9025
batch: 155/1000 	    computation time/batch: 53.9054sec	    number of points: 15
	    median distance2db: 0.00024306774139404297
computation DB time/batch: 53.9174
batch: 156/1000 	    computation time/batch: 53.9205sec	    number of points: 15
	    median distance2db: 0.003953695297241211
computation DB time/batch: 53.9448
batch: 157/1000 	    computation time/batch: 53.9493sec	    number of points: 6
	    median distance2db: 0.0005315542221069336
computation DB time/batch: 53.9502
batch: 158/1000 	    computation time/batch: 53.9530sec	    number of points: 9
	    median distance2db: 9.274482727050781e-05
computation DB time/batch: 53.9686
batch: 159/1000 	    computation time/batch: 53.9718sec	    number of points: 17
	    median distance2db: 0.0010788440704345703
computation DB time/batch

computation DB time/batch: 53.9896
batch: 205/1000 	    computation time/batch: 53.9925sec	    number of points: 17
	    median distance2db: 0.003932952880859375
computation DB time/batch: 53.9923
batch: 206/1000 	    computation time/batch: 53.9953sec	    number of points: 18
	    median distance2db: 0.002730727195739746
computation DB time/batch: 53.9532
batch: 207/1000 	    computation time/batch: 53.9562sec	    number of points: 17
	    median distance2db: 0.002228379249572754
computation DB time/batch: 53.9197
batch: 208/1000 	    computation time/batch: 53.9228sec	    number of points: 9
	    median distance2db: 0.004935622215270996
computation DB time/batch: 53.9158
batch: 209/1000 	    computation time/batch: 53.9185sec	    number of points: 14
	    median distance2db: 0.0012112855911254883
computation DB time/batch: 53.9687
batch: 210/1000 	    computation time/batch: 53.9716sec	    number of points: 17
	    median distance2db: 0.001924753189086914
computation DB time/batch: 5

...

computation DB time/batch: 53.9768
batch: 715/1000 	    computation time/batch: 53.9804sec	    number of points: 7
	    median distance2db: 0.002598285675048828
computation DB time/batch: 53.9616
batch: 716/1000 	    computation time/batch: 53.9647sec	    number of points: 14
	    median distance2db: 0.002942800521850586
computation DB time/batch: 53.9670
batch: 717/1000 	    computation time/batch: 53.9711sec	    number of points: 7
	    median distance2db: 0.002062082290649414
computation DB time/batch: 53.9680
batch: 718/1000 	    computation time/batch: 53.9709sec	    number of points: 18
	    median distance2db: 0.0009320974349975586
computation DB time/batch: 53.9791
batch: 719/1000 	    computation time/batch: 53.9823sec	    number of points: 14
	    median distance2db: 0.0022431612014770508
computation DB time/batch: 53.9592
batch: 720/1000 	    computation time/batch: 53.9626sec	    number of points: 14
	    median distance2db: 0.0031867027282714844
computation DB time/batch: 

computation DB time/batch: 53.9359
batch: 766/1000 	    computation time/batch: 53.9389sec	    number of points: 13
	    median distance2db: 0.0006694793701171875
computation DB time/batch: 53.9259
batch: 767/1000 	    computation time/batch: 53.9286sec	    number of points: 10
	    median distance2db: 0.00043702125549316406
computation DB time/batch: 53.9533
batch: 768/1000 	    computation time/batch: 53.9564sec	    number of points: 13
	    median distance2db: 0.0028668642044067383
computation DB time/batch: 53.9853
batch: 769/1000 	    computation time/batch: 53.9880sec	    number of points: 15
	    median distance2db: 0.004914641380310059
computation DB time/batch: 54.0073
batch: 770/1000 	    computation time/batch: 54.0110sec	    number of points: 6
	    median distance2db: 0.001940608024597168
computation DB time/batch: 53.9640
batch: 771/1000 	    computation time/batch: 53.9670sec	    number of points: 10
	    median distance2db: 0.003809809684753418
computation DB time/batch

computation DB time/batch: 53.9391
batch: 817/1000 	    computation time/batch: 53.9422sec	    number of points: 15
	    median distance2db: 0.0028723478317260742
computation DB time/batch: 53.9158
batch: 818/1000 	    computation time/batch: 53.9185sec	    number of points: 13
	    median distance2db: 0.0016623735427856445
computation DB time/batch: 53.9028
batch: 819/1000 	    computation time/batch: 53.9061sec	    number of points: 17
	    median distance2db: 0.0010516643524169922
computation DB time/batch: 53.9000
batch: 820/1000 	    computation time/batch: 53.9027sec	    number of points: 14
	    median distance2db: 0.00352632999420166
computation DB time/batch: 53.9190
batch: 821/1000 	    computation time/batch: 53.9221sec	    number of points: 14
	    median distance2db: 0.006494641304016113
computation DB time/batch: 53.9647
batch: 822/1000 	    computation time/batch: 53.9676sec	    number of points: 9
	    median distance2db: 9.691715240478516e-05
computation DB time/batch:

computation DB time/batch: 53.9349
batch: 868/1000 	    computation time/batch: 53.9380sec	    number of points: 16
	    median distance2db: 0.0012805461883544922
computation DB time/batch: 53.9844
batch: 869/1000 	    computation time/batch: 53.9873sec	    number of points: 15
	    median distance2db: 0.0018360614776611328
computation DB time/batch: 53.9959
batch: 870/1000 	    computation time/batch: 53.9989sec	    number of points: 15
	    median distance2db: 0.0010439157485961914
computation DB time/batch: 53.9833
batch: 871/1000 	    computation time/batch: 53.9860sec	    number of points: 10
	    median distance2db: 0.0037169456481933594
computation DB time/batch: 53.9556
batch: 872/1000 	    computation time/batch: 53.9590sec	    number of points: 12
	    median distance2db: 0.00040841102600097656
computation DB time/batch: 53.9285
batch: 873/1000 	    computation time/batch: 53.9316sec	    number of points: 19
	    median distance2db: 0.0011867284774780273
computation DB time/b

KeyboardInterrupt: 

In [ ]:
db_sample_df.to_csv('computed_codim_3points_1M.csv')

In [14]:
print(f'computed_points_1M_{10:04d}.csv')

computed_points_1M_0010.csv


In [49]:
print(f'Number of points: {db_sample_df.shape}')

fig = px.scatter_3d(db_sample_df[:1000], x='x0', y='x1', z='x2')
fig.show()

Number of points: (52, 3)


In [ ]:
db_sample_df.to_csv('DB_3d_MNIST.csv')

In [51]:
print(f'Number of points: {db_sample_df.shape}')

fig = px.scatter_3d(db_sample_df[:1000], x='x0', y='x1', z='x2')
fig.show()

Number of points: (106, 3)


In [41]:
embedding_model_multi = nn.Sequential(model_auto.decoder,
                    LambdaLayer(lambda x: x.reshape(-1, 1, 28, 28)),
                    model)

In [44]:
embedding_model_multi.to('cpu')

Sequential(
  (0): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Tanh()
  )
  (1): LambdaLayer()
  (2): Net(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (dropout1): Dropout(p=0.25, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
    (fc1): Linear(in_features=9216, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [45]:
db_tensor = torch.tensor(db_sample_df.values)
db_tensor_logit = embedding_model_multi(db_tensor)
max_top2 = torch.topk(db_tensor_logit, 2).indices.max(axis=1)
db_class = {}
for digit_class in range(10):
    db_class[digit_class] = {}
    db_class[digit_class]['tensor'] = max_top2.values == digit_class
    db_class[digit_class]['idx'] = torch.nonzero(db_class[digit_class]['tensor'])

In [46]:
db_sample_class_df = pd.DataFrame(columns=['class'])

for digit_class in range(10):
    db_sample_class_df = db_sample_class_df.append(pd.DataFrame(db_class[digit_class]['idx'][:,0].shape[0]*[digit_class], columns=['class'], index=db_class[digit_class]['idx'][:,0].numpy()))

In [ ]:
db_sample_normal_df = pd.DataFrame(columns = ["dx"+str(i) for i in range(3)])

for digit_class in range(10):
    x = db_tensor[db_class[digit_class]['tensor']]
    if x.shape[0] != 0:
        x.requires_grad = False
        delta = torch.zeros_like(x, requires_grad=True)
        loss = torch.sum(embedding_model_multi(x + delta)[:, digit_class])
        loss.backward()
        x_gradient = delta.grad.detach().numpy()
        x_norm = x_gradient/np.sqrt((x_gradient**2).sum(axis=1, keepdims=True))

        db_sample_normal_df = db_sample_normal_df.append(
            pd.DataFrame(data=x_norm,
                columns = ["dx"+str(i) for i in range(3)],
                index=db_class[digit_class]['idx'].numpy()[:,0])
        )

In [ ]:
db_mesh_input = pd.concat([db_sample_df, db_sample_normal_df, db_sample_class_df], axis=1, join='inner')